## Dataset & DataLoader 살표보기

-   Pytorch에서 배치 크기만큼 데이터를 조절하기 위한 메카니즘
-   Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
-   DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피쳐와 타겟을 추출하여 전달


### [1] 모듈 로딩 및 데이터 준비 <hr>


In [2]:
### 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

import torchvision

print(torchvision.__version__)


0.17.1


In [3]:
### 데이터 준비

x_data = torch.IntTensor(
    [[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60, 70]]
)
y_data = torch.IntTensor([[20], [30], [40], [50], [60]])

print(f"x_data = > {x_data.shape}, {x_data.ndim}D")
print(f"y_data = > {y_data.shape}, {y_data.ndim}D")


x_data = > torch.Size([5, 3]), 2D
y_data = > torch.Size([5, 1]), 2D


### [2] 데이터셋 생성 <hr>


#### [2-1] TensorDataset 활용 : Dataset의 sub_class


In [4]:
# TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset


In [5]:
dataset = TensorDataset(
    x_data, y_data
)  # x_data, y_data shape[0]이 다르면 dataset 생성 안 됨
dataset


In [6]:
dataset.tensors


(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20],
         [30],
         [40],
         [50],
         [60]], dtype=torch.int32))

In [7]:
### __getitem__() 메서드 호출
dataset[0]


(tensor([10, 20, 30], dtype=torch.int32), tensor([20], dtype=torch.int32))

In [8]:
len(dataset)


5

#### [2-2] 사용자정의 데이터셋 생성


In [9]:
### 데이터 준비
filename = "../data/text/iris.csv"

irisDF = pd.read_csv(filename)
irisDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [11]:
irisNP = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=(0, 1, 2, 3))
irisNP[:2]


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [13]:
## 데이터의 타입 체크
type(irisDF), type(irisNP), irisDF.__class__.__name__, irisNP.__class__.__name__


(pandas.core.frame.DataFrame, numpy.ndarray, 'DataFrame', 'ndarray')

In [14]:
if irisDF.__class__.__name__ == "DataFrame":
    print("DF")
else:
    print("------")


DF


In [21]:
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame), isinstance(
    irisNP, np.ndarray
)


(True, False, True)

In [20]:
isinstance([10], list), isinstance({"A": 22}, list)


(True, False)

In [23]:
### 사용자정의 DataSet 클래스
# - 데이터의 Tensor 변환
class DLDataset(Dataset):
    # 초기화 콜백함수(callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        # x,y 데이터 ==> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data

        # ndarray ==> tensor
        self.feature = torch.tensor(x_data)
        self.target = torch.tensor(y_data)

    # 데이터셋의 갯수 체크 콜백함수(callback function)
    def __len__(self):
        return self.target.shape[0]

    # 특정 인덱스 데이터 + 라벨 반환 콜백함수(callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]


In [26]:
## 피쳐와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1]]

print(f"featureDF => {featureDF.shape}, {featureDF.ndim}D")
print(f"targetDF  => {targetDF.shape}, {targetDF.ndim}D")


featureDF => (150, 4), 2D
targetDF  => (150,), 1D


In [31]:
# object 타입 타겟 ===> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetDF)
targetNP = targetNP.reshape(-1, 1)

print(targetNP.shape, targetNP.ndim)


(150, 1) 2


In [33]:
# 데이터셋 생성 -> DF, NP
my_dataset = DLDataset(featureDF, targetNP)


In [37]:
my_dataset[0], featureDF.iloc[0], targetDF[0]


((tensor([5.1000, 3.5000, 1.4000, 0.2000], dtype=torch.float64),
  tensor([0], dtype=torch.int32)),
 sepal_length    5.1
 sepal_width     3.5
 petal_length    1.4
 petal_width     0.2
 Name: 0, dtype: float64,
 'setosa')

In [40]:
# 데이터셋 생성 -> NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)
my_dataset2[0]


(tensor([5.1000, 3.5000, 1.4000, 0.2000], dtype=torch.float64),
 tensor([0], dtype=torch.int32))